In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
# Loading the dataset
def loaddata(filename):
    df = pd.read_csv(f'/content/drive/MyDrive/data-brm/{filename}.csv', sep=',', encoding='latin-1')
    return df

def loaddata2():
    df = pd.read_csv(f'/content/drive/MyDrive/cleaned_book_data.csv', sep=',', encoding='latin-1')
    return df

books   = loaddata2()
ratings = loaddata("ratings")

In [ ]:
rating_users = ratings['user_id'].value_counts().reset_index().\
               rename({'index':'user_id','user_id':'rating'}, axis=1)

rating_books = ratings['book_id'].value_counts().reset_index().\
               rename({'index':'book_id','book_id':'rating'}, axis=1)


ratings = ratings[ratings['user_id'].isin(rating_users[rating_users['rating']>5]['user_id'])]
ratings = ratings[ratings['book_id'].isin(rating_books[rating_books['rating']> 2000]['book_id'])]

temp = ratings.head(650000)
ratings = temp

In [ ]:
import torch

!pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
!pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 55.4 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 44.2 MB/s eta 0:00:00
Looking in links: https://data.pyg.org/whl/torch-2.0.1+cu118.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 37.1 MB/s eta 0:00:00
  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-qontr7kx
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-qontr7kx
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 080a6e9246c040ac77516862b5729caf05996641
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for torch_geometric: filename=torch_geometric-2.4.0-py3-none-any.whl size=10

In [ ]:
from sklearn import model_selection, metrics, preprocessing
import numpy as np

In [ ]:
# import required modules
import random
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import model_selection, metrics, preprocessing
import copy

import torch
from torch import nn, optim, Tensor

from torch_sparse import SparseTensor, matmul

from collections import defaultdict

from torch_geometric.utils import structured_negative_sampling
from torch_geometric.data import download_url, extract_zip
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.typing import Adj
import torch.nn.functional as F

In [ ]:
# split the edges of the graph using a 80/10/10 train/validation/test split
# num_users, num_books = len(user_mapping), len(book_mapping)

num_users = len(ratings['user_id'].unique())
num_books = len(ratings['book_id'].unique())

print(f"num_users {num_users}, num_books {num_books}")

num_users 16271, num_books 506


In [ ]:
from google.colab import files

# Use files.upload() to upload a file from your local system
uploaded = files.upload()

# map_location=torch.device('cpu')
# Load the uploaded model file
model_path = 'trained_graph_rec_model.pth'  # Specify the filename
model = torch.load(model_path)


Saving trained_graph_rec_model.pth to trained_graph_rec_model.pth


In [ ]:
import joblib
# Upload the file from your local machine to Colab
uploaded = files.upload()

# Check if the file was successfully uploaded
if 'label_encoder_books.pkl' in uploaded:
    print("File uploaded successfully!")

# Load the label encoder from the uploaded file
loaded_label_encoder = joblib.load('label_encoder_books.pkl')
lbl_books = loaded_label_encoder

# # Now you can use loaded_label_encoder to transform data as needed
# # For example, if you want to transform a list of labels:
# encoded_labels = loaded_label_encoder.transform(['label1', 'label2', 'label3'])

# # If you want to inverse transform encoded labels to get the original labels:
# original_labels = loaded_label_encoder.inverse_transform(encoded_labels)


Saving label_encoder_books.pkl to label_encoder_books.pkl
File uploaded successfully!


In [ ]:
model


In [ ]:
!pip install torchsummary


In [ ]:
from torchsummary import summary
summary(model, input_size=(num_users, num_books))


In [ ]:
def recommend_books(user_id, model, lbl_books, num_recommendations, device):
    # Assuming 'user_id' is the user for whom you want personalized recommendations
    user_embedding = model['users_emb.weight'][user_id].to(device)



    # Get all item embeddings
    all_item_embeddings = model['items_emb.weight'].to(device)

    # print("all_item_embeddings:  ",all_item_embeddings)

    # Compute scores for all items using dot product
    scores = torch.mm(all_item_embeddings, user_embedding.view(-1, 1)).squeeze().cpu().detach().numpy()

    # print("scores: ",scores)

    # Sort the items by their scores in descending order
    sorted_indices = np.argsort(scores, axis=0)[::-1]

    # print("sorted_indices: ",sorted_indices)

    # # Get the top 'num_recommendations' book indices
    top_indices = sorted_indices[:num_recommendations]

    # # Map the book indices back to their original labels
    recommended_books = lbl_books.classes_[top_indices]

    # Create a list of tuples containing book and rating
    recommendations_with_ratings = [(book, score) for book, score in zip(recommended_books, scores[top_indices])]
    print("ratings: ",recommendations_with_ratings)

    return recommended_books,recommendations_with_ratings

In [ ]:
user_id = 12
num_recommendations = 10

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

recommended_books,recommendations_with_ratings = recommend_books(user_id, model, lbl_books, num_recommendations, device)

print(f"Top {num_recommendations} recommended books with ratings for user {user_id}:")
for i, (book_id, rating) in enumerate(zip(recommended_books, recommendations_with_ratings)):
    book_info = books[books['book_id'] == book_id]
    title = book_info['title'].values[0]
    # print(f"{rating[1]:.16f}")
    print(f"{i+1}. Book ID: {book_id}, Title: {title}")

ratings:  [(271, 2.3407939e-08), (232, 2.1333932e-08), (241, 1.9653946e-08), (130, 1.6812026e-08), (516, 1.5805895e-08), (152, 1.5370253e-08), (44, 1.4397287e-08), (705, 1.272403e-08), (182, 1.1681058e-08), (239, 1.047633e-08)]
Top 10 recommended books with ratings for user 12:
1. Book ID: 271, Title: Wizardology
2. Book ID: 232, Title: Starting Out with C++: From Control Structures Through Objects
3. Book ID: 241, Title: The Burning City
4. Book ID: 130, Title: The Well-Read Witch: Essential Books for Your Magickal Library
5. Book ID: 516, Title: Tales Out of Dunwich
6. Book ID: 152, Title: Neuroscience and Philosophy: Brain, Mind, and Language
7. Book ID: 44, Title: The Pony and the Bear
8. Book ID: 705, Title: Tides of Consent: How Public Opinion Shapes American Politics
9. Book ID: 182, Title: Through No Fault of My Own: A Girl's Diary of Life on Summit Avenue in the Jazz Age
10. Book ID: 239, Title: Barren Lands: An Epic Search For Diamonds in the North American Artic
